# OpenAI API Related

> 帮助OPENAI API 推理有关的函数

In [12]:
#| default_exp llm_api

In [13]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

pandas格式检查

In [14]:
#| export
import pandas as pd

def make_info_df(df, n_samples=2):
    """
    构造包含 column_name, dtype 和前 n_samples 行样本值的表格
    """
    info_df = pd.DataFrame({
        'column_name': df.columns,
        'dtype': df.dtypes.values
    })
    # 添加前 n_samples 行样本值
    for i in range(n_samples):
        col = f'sample{i+1}'
        info_df[col] = df.iloc[i].values
    return info_df

In [15]:
#| export
import pydantic

def get_pydantic_version():
    """
    判断当前使用的是 Pydantic 1 还是 Pydantic 2
    :return: 版本号 1 或 2
    """
    try:
        if pydantic.VERSION.startswith('1'):
            return 1
        elif pydantic.VERSION.startswith('2'):
            return 2
        else:
            raise ValueError(f"不支持的 Pydantic 版本: {pydantic.VERSION}")
    except AttributeError:
        # Pydantic 1 可能没有 VERSION 属性，通过其他方式判断
        try:
            from pydantic.main import ModelMetaclass
            return 1
        except ImportError:
            return 2

In [16]:
get_pydantic_version()

2

In [17]:
#| export
import re
from urllib.parse import urlparse
import os

def is_url(s: str) -> bool:
    """判断是否为URL（包含http/https/ftp/file等协议）"""
    # 1. 语法校验：协议前缀 + ://（file:// 也符合）
    url_pattern = r'^[a-zA-Z][a-zA-Z0-9+.-]*://[^\s]*$'
    if not re.match(url_pattern, s.strip()):
        return False
    # 2. 解析协议，确认是合法URL协议
    parsed = urlparse(s)
    valid_schemes = {'http', 'https', 'ftp', 'ftps', 'sftp', 'ssh', 'telnet', 'file'}
    return parsed.scheme.lower() in valid_schemes

def is_local_file_path(s: str) -> bool:
    """判断是否为本地文件路径（非URL格式）"""
    # 排除URL（避免与file://混淆）
    if is_url(s):
        return False
    # 检查是否符合操作系统的文件路径格式
    # 简化判断：包含路径分隔符，或符合盘符（Windows）/根目录（Linux/macOS）特征
    s_stripped = s.strip()
    if not s_stripped:
        return False
    # Windows路径特征：盘符（如C:）+ 反斜杠或斜杠
    windows_pattern = r'^[a-zA-Z]:[\\/].*'
    # Linux/macOS路径特征：根目录/或相对路径./../
    unix_pattern = r'^(/|\./|\.\./).*'
    # 还可以通过尝试解析路径是否合法进一步验证（可选）
    try:
        # 尝试规范化路径（若报错则不是有效路径）
        os.path.normpath(s_stripped)
        return re.match(windows_pattern, s_stripped) is not None or \
               re.match(unix_pattern, s_stripped) is not None
    except:
        return False

In [18]:
# 测试用例
test_cases = [
    # URL
    "https://www.baidu.com",    # URL → True
    "ftp://ftp.example.com/file.zip",  # URL → True
    "file:///C:/Users/test.txt",  # 本地文件URL → True（is_url返回True）
    # 本地文件路径
    "C:\\Users\\test.txt",     # Windows本地路径 → True
    "/home/user/docs.pdf",     # Linux本地路径 → True
    "./data/report.csv",       # 相对路径 → True
    # 其他
    "www.baidu.com",           # 既非URL也非路径 → 两者都False
    "tel:123456",              # URI但非URL/路径 → 两者都False
]

for case in test_cases:
    url_flag = is_url(case)
    path_flag = is_local_file_path(case)
    print(f"{case}:")
    print(f"  是URL: {url_flag}")
    print(f"  是本地文件路径: {path_flag}\n")

https://www.baidu.com:
  是URL: True
  是本地文件路径: False

ftp://ftp.example.com/file.zip:
  是URL: True
  是本地文件路径: False

file:///C:/Users/test.txt:
  是URL: True
  是本地文件路径: False

C:\Users\test.txt:
  是URL: False
  是本地文件路径: True

/home/user/docs.pdf:
  是URL: False
  是本地文件路径: True

./data/report.csv:
  是URL: False
  是本地文件路径: True

www.baidu.com:
  是URL: False
  是本地文件路径: False

tel:123456:
  是URL: False
  是本地文件路径: False



### 辅助函数：基于文件内容检测扩展名
需要先安装: pip install filetype
如果没有 filetype，这个函数可以回退到默认行为或者报错，这里演示最稳健的 filetype 方式。

In [29]:
#| export
import struct
import aiofiles
from pathlib import Path
from typing import Optional

# Try importing filetype, treat as optional dependency
try:
    import filetype
except ImportError:
    filetype = None

async def detect_and_rename(
    fpath: Path, # The current file path
    verbose: bool = False # Whether to print log messages
) -> Path:
    "Async reads file magic bytes (using filetype or fallback) and renames if needed."
    
    final_path = fpath
    if not fpath.exists(): return fpath

    # Common magic numbers mapping (Fallback if filetype is missing/fails)
    signatures = {
        b'\xFF\xD8\xFF': '.jpg',
        b'\x89PNG\r\n\x1a\n': '.png',
        b'GIF87a': '.gif',
        b'GIF89a': '.gif',
        b'%PDF': '.pdf',
        b'\x1aE\xdf\xa3': '.webm', 
    }

    try:
        # Async read the first 2KB (enough for filetype and most headers)
        head = b""
        async with aiofiles.open(fpath, 'rb') as f:
            head = await f.read(2048)
            
        ext = None
        
        # --- Strategy 1: Use 'filetype' library if available ---
        if filetype:
            kind = filetype.guess(head)
            if kind:
                ext = '.' + kind.extension
        
        # --- Strategy 2: Fallback manual detection ---
        if ext is None:
            # Check dictionary signatures
            for sig, likely_ext in signatures.items():
                if head.startswith(sig):
                    ext = likely_ext
                    break
            
            # Check MP4/MOV (ftyp at index 4)
            if ext is None and len(head) > 12:
                if head[4:8] == b'ftyp':
                    ext = '.mp4'

        # --- Rename Logic ---
        if ext:
            if fpath.suffix.lower() != ext:
                new_path = fpath.with_suffix(ext)
                # Rename is an OS metadata op, typically fast enough to be sync
                fpath.rename(new_path)
                final_path = new_path
                if verbose: print(f"🔄 Renamed: {fpath.name} -> {final_path.name}")
            elif verbose:
                print(f"✅ Extension verified: {ext}")
        else:
            if verbose: print("⚠️ Could not detect file type, keeping original name.")

    except Exception as e:
        if verbose: print(f"⚠️ Extension detection failed: {e}")
        
    return final_path

In [30]:
import asyncio

async def test_detection():
    # 1. Create a dummy file without extension (Mocking a PNG)
    # PNG Signature: 89 50 4E 47 0D 0A 1A 0A
    png_header = b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR' 
    dummy_path = Path("test_image_no_ext")
    
    async with aiofiles.open(dummy_path, 'wb') as f:
        await f.write(png_header)
        
    print(f"🧪 Created dummy file: {dummy_path}")

    try:
        # 2. Run async detection
        # Note: If you don't have filetype installed, this tests the fallback logic.
        # If you do, it tests the library logic. Both should result in .png
        fixed_path = await detect_and_rename(dummy_path, verbose=True)
        
        # 3. Assertions
        assert fixed_path.suffix == '.png', f"Expected .png, got {fixed_path.suffix}"
        assert fixed_path.exists()
        assert not dummy_path.exists()
        print("✅ Async type detection passed")
        
    finally:
        # Cleanup
        if fixed_path.exists(): fixed_path.unlink()
        if dummy_path.exists(): dummy_path.unlink()

await test_detection()

🧪 Created dummy file: test_image_no_ext
🔄 Renamed: test_image_no_ext -> test_image_no_ext.png
✅ Async type detection passed


In [19]:
# pip install aiofiles

In [31]:
#| export
import aiohttp
import aiofiles
import tempfile
from pathlib import Path
from typing import Optional, Union
from urllib.parse import urlparse, unquote

async def download_file(
    video_url: str, # The URL to download
    verbose: bool = False, # Toggle verbose logging
    target: Optional[Union[str, Path]] = None # Target directory or full filepath
) -> Optional[str]:
    "Async downloads a file, then auto-detects and corrects the file extension."
    
    save_path: Path = Path()
    
    try:
        # --- 1. Setup Path ---
        parsed_url = urlparse(video_url)
        url_filename = Path(unquote(parsed_url.path)).name
        if not url_filename: url_filename = "downloaded_file.tmp"

        if target:
            target_path = Path(target)
            if target_path.is_dir():
                save_path = target_path / url_filename
            else:
                save_path = target_path
                save_path.parent.mkdir(parents=True, exist_ok=True)
        else:
            suffix = Path(url_filename).suffix
            with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
                save_path = Path(tmp.name)

        if verbose: print(f"⬇️ Start Download: {video_url} -> {save_path}")

        # --- 2. Async Download ---
        timeout = aiohttp.ClientTimeout(total=600)
        async with aiohttp.ClientSession() as session:
            async with session.get(video_url, timeout=timeout) as response:
                if response.status != 200:
                    if verbose: print(f"❌ Status {response.status}")
                    if save_path.exists(): save_path.unlink()
                    return None

                async with aiofiles.open(save_path, "wb") as f:
                    async for chunk in response.content.iter_chunked(1024 * 1024):
                        await f.write(chunk)

        # --- 3. Async Type Detection & Rename ---
        # Await the new async helper function
        final_path = await detect_and_rename(save_path, verbose=verbose)
        
        return str(final_path.absolute())

    except Exception as e:
        if verbose: print(f"❌ Error: {e}")
        if save_path and save_path.exists():
            try: save_path.unlink()
            except OSError: pass
        return None

In [32]:
import asyncio

video_url = "http://www.w3school.com.cn/example/html5/mov_bbb.mp4"  # 替换为实际URL
input_video = await download_file(video_url)
if input_video:
    print(f"下载完成，本地路径：{input_video}")
        # 使用完成后手动删除临时文件（可选）
        # os.unlink(video_path)

下载完成，本地路径：/tmp/tmpbusenzhs.mp4


In [33]:
import asyncio

video_url = "http://www.w3school.com.cn/example/html5/mov_bbb.mp4"  # 替换为实际URL
input_video = await download_file(video_url, True, target="./test/vie.any.suffix")
if input_video:
    print(f"下载完成，本地路径：{input_video}")
        # 使用完成后手动删除临时文件（可选）
        # os.unlink(video_path)
    os.remove(input_video)

⬇️ Start Download: http://www.w3school.com.cn/example/html5/mov_bbb.mp4 -> test/vie.any.suffix
🔄 Renamed: vie.any.suffix -> vie.any.mp4
下载完成，本地路径：/mnt/bn/pippit-omni-eval/mlx/users/yecanming/repos/BoGuan_YueQu/third_party/infras/ScholarlyInfrastructure/src/notebooks/test/vie.any.mp4


In [34]:
async def test_download_integration():
    # Test with a known image URL that has an extension, 
    # but we will force the target to NOT have one to trigger the rename logic.
    url = "https://www.w3schools.com/w3css/img_lights.jpg"
    
    with tempfile.TemporaryDirectory() as td:
        # Force a target path without extension
        target_path = Path(td) / "mystery_file"
        
        print(f"🚀 Downloading to: {target_path}")
        result = await download_file(url, verbose=True, target=target_path)
        
        if result:
            res_path = Path(result)
            print(f"📂 Final Path: {res_path}")
            
            # Check if it recognized it as a jpg and renamed it
            assert res_path.suffix == '.jpg', "Should detect .jpg extension"
            assert res_path.name != "mystery_file", "Filename should have changed"
            assert res_path.exists()
            print("✅ Download and Rename Integration Test Passed")
        else:
            print("❌ Download failed (check network)")

await test_download_integration()

🚀 Downloading to: /tmp/tmpv4kgrn36/mystery_file
⬇️ Start Download: https://www.w3schools.com/w3css/img_lights.jpg -> /tmp/tmpv4kgrn36/mystery_file
🔄 Renamed: mystery_file -> mystery_file.jpg
📂 Final Path: /tmp/tmpv4kgrn36/mystery_file.jpg
✅ Download and Rename Integration Test Passed


In [40]:
#| export
import base64
import os
import asyncio

try:
    import aiofiles
except ImportError:
    print("请先安装 'aiofiles' 库: !pip install aiofiles")

async def local_video_to_base64_uri(file_path: str) -> str:
    """
    异步地将本地视频文件转换为 Base64 编码的数据 URI。
    
    格式遵循: data:video/<视频格式>;base64,<Base64编码>，其中，
    视频格式: 支持 mp4, avi, mov。
    Base64编码: 视频文件的 Base64 编码。

    Args:
        file_path: 本地视频文件的路径。

    Returns:
        一个字符串，包含了视频的 Base64 编码数据 URI。
        
    Raises:
        ValueError: 如果视频格式不是 'mp4', 'avi', 或 'mov' 之一。
        FileNotFoundError: 如果在 file_path 指定的路径下找不到文件。
    """
    supported_formats = {'mp4', 'avi', 'mov'}
    
    # 从路径中获取文件扩展名作为视频格式
    file_extension = file_path.split('.')[-1].lower()

    if file_extension not in supported_formats:
        raise ValueError(f"不支持的视频格式: '{file_extension}'。支持的格式为: {', '.join(supported_formats)}")

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件未找到: {file_path}")

    # 异步读取文件
    async with aiofiles.open(file_path, 'rb') as video_file:
        video_bytes = await video_file.read()
    
    # 进行 Base64 编码
    base64_encoded_video = base64.b64encode(video_bytes).decode('utf-8')
    
    # 拼接成最终的 Data URI 字符串
    return f"data:video/{file_extension};base64,{base64_encoded_video}"



In [41]:
"""一个运行示例的异步函数"""
# 创建一个用于演示的虚拟视频文件
# 在实际使用中，请将此路径替换为您的视频文件路径
dummy_video_path = input_video or "/tmp/test.mp4"
if not os.path.exists(dummy_video_path):
    print(f"正在创建虚拟文件用于演示: {dummy_video_path}")
    with open(dummy_video_path, "wb") as f:
        # 这是一个合法的、但非常小的 MP4 文件内容
        f.write(b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00isommp42')

try:
    print(f"正在转换文件: {dummy_video_path}")
    data_uri = await local_video_to_base64_uri(dummy_video_path)
    print("转换成功！")
    # 通常 Data URI 会非常长，这里只打印前100个字符
    print(f"生成的数据 URI (前100个字符): {data_uri[:100]}...")
except (ValueError, FileNotFoundError, NameError) as e:
    print(f"发生错误: {e}")
finally:
    # 示例运行后，您可以选择保留或删除这个虚拟文件
    if os.path.exists(dummy_video_path):
        print(f"示例文件 '{dummy_video_path}' 已保留，您可以检查或手动删除。")


正在转换文件: /tmp/tmp8gzgpnrt.mp4
转换成功！
生成的数据 URI (前100个字符): data:video/mp4;base64,AAAAHGZ0eXBtcDQyAAAAAG1wNDJpc29tYXZjMQAAAIRmcmVlAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
示例文件 '/tmp/tmp8gzgpnrt.mp4' 已保留，您可以检查或手动删除。


In [42]:
#| export
import re
from typing import Optional, Tuple
def separate_think_and_other(text: str) -> Tuple[Optional[str], str]:
    """
    从文本中分离 <think> 标签内容和其余内容。

    Args:
        text: 包含或不包含 <think> 标签的原始字符串。

    Returns:
        一个元组，包含两部分：
        - 第一个元素是所有 <think> 标签内内容的合并字符串（以换行符分隔），如果没有则为 None。
        - 第二个元素是去除 <think> 标签后剩余内容的合并字符串。
    """
    # 使用 re.findall 找到所有 <think> 标签的内容
    # re.DOTALL 标志让 '.' 可以匹配包括换行符在内的任何字符
    think_parts = re.findall(r'<think>(.*?)</think>', text, re.DOTALL)
    
    if think_parts:
        # 合并所有 <think> 标签内的内容，使用换行符分隔
        think_content = "\n".join(part.strip() for part in think_parts)
    else:
        think_content = None

    # 使用 re.split 来获取 <think> 标签外的内容
    other_parts = re.split(r'<think>.*?</think>', text, flags=re.DOTALL)
    # 合并所有标签外的内容，并清理首尾及中间多余的空白
    other_content = "\n".join(part.strip() for part in other_parts if part.strip())

    return think_content, other_content

In [43]:
# 测试用例
test_cases = [
    "这是一个测试字符串",
    "普通文本<think>这是思考内容</think>其他部分",
    "<think>只有思考内容</think>",
    "无思考内容的普通文本",
    "多思考部分<think>思考1</think>中间文本<think>思考2</think>结尾文本",
    """
    <think>
Got it, let's tackle this problem. First, the user is asking about what the person is doing, whether the sound is pleasant, and if the drawing looks good. They want the output in a specific JSON format called RaterResult with description, speech_to_text, draw_quality, and sound_quality.

First, analyze the video: The video shows someone using a stylus on a tablet to draw a guitar. So the description should capture that. Then, the speech: the person says "Hello, take a look at what I'm drawing." So speech_to_text is that sentence. Then for draw_quality: the drawing is a simple, clean guitar illustration, which is decent but not overly complex, so maybe 4 stars. Sound quality: the voice is clear, no background noise, so 5 stars.

Let's check each part:

Description: "A person is using a stylus to draw a guitar illustration on a tablet."

Speech to text: "Hello, take a look at what I'm drawing."

Draw quality: The drawing is clear, well-defined lines, simple but neat. So 4 stars (since it's good but maybe not perfect for a highly detailed work, but the question is "画得好看不?" so 4 is reasonable).

Sound quality: The voice is clear, no distortion, so 5 stars.

Now structure into JSON as per schema.
</think>

{
    "description": "A person is using a stylus to draw a guitar illustration on a tablet.",
    "speech_to_text": "Hello, take a look at what I'm drawing.",
    "draw_quality": 4,
    "sound_quality": 5
}
    """
]

for i, case in enumerate(test_cases, 1):
    wrapped, remaining = separate_think_and_other(case)
    print(f"测试用例 {i}:")
    print(f"原始字符串: {case!r}")
    print(f"包裹内容合并: {wrapped!r}")
    print(f"剩余部分合并: {remaining!r}")
    print("-" * 50)

测试用例 1:
原始字符串: '这是一个测试字符串'
包裹内容合并: None
剩余部分合并: '这是一个测试字符串'
--------------------------------------------------
测试用例 2:
原始字符串: '普通文本<think>这是思考内容</think>其他部分'
包裹内容合并: '这是思考内容'
剩余部分合并: '普通文本\n其他部分'
--------------------------------------------------
测试用例 3:
原始字符串: '<think>只有思考内容</think>'
包裹内容合并: '只有思考内容'
剩余部分合并: ''
--------------------------------------------------
测试用例 4:
原始字符串: '无思考内容的普通文本'
包裹内容合并: None
剩余部分合并: '无思考内容的普通文本'
--------------------------------------------------
测试用例 5:
原始字符串: '多思考部分<think>思考1</think>中间文本<think>思考2</think>结尾文本'
包裹内容合并: '思考1\n思考2'
剩余部分合并: '多思考部分\n中间文本\n结尾文本'
--------------------------------------------------
测试用例 6:
原始字符串: '\n    <think>\nGot it, let\'s tackle this problem. First, the user is asking about what the person is doing, whether the sound is pleasant, and if the drawing looks good. They want the output in a specific JSON format called RaterResult with description, speech_to_text, draw_quality, and sound_quality.\n\nFirst, analyze the video: The vide

In [10]:
#| export
import re
from typing import Optional

def extract_code_content(text: str, target_lang: Optional[str] = None) -> str:
    """
    从字符串中提取第一个 Markdown 代码块的内容。
    如果没有代码块，则直接返回原文本。
    """
    if target_lang is None:
        pattern = re.compile(r"```(?:[a-zA-Z0-9_+-]*)\n(.*?)```", re.DOTALL)
    else:
        pattern = re.compile(rf"```{target_lang}\n(.*?)```", re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(1).strip()
    else:
        return text.strip()

In [9]:
test_cases = {
    "1. 有语言标识的代码块": """这里是描述
```python
print("Hello, world!")
x = 1 + 2
```

这是结尾""",

"2. 没有语言标识的代码块": """前面的话
```

```
for i in range(3):
    print(i)
```

后面的话""",

"3. 只有普通文本，没有代码块": """print('Just plain text, no markdown code block')""",

"4. 多个代码块（只提取第一个）": """这是第一个

```python
a = 10
```

这是第二个

```bash
echo "hi"
```

结束""",

"5. 空字符串": "",

"6. 代码块内含空行": """Example:
```

```python
def foo():
    
    return 42
```

End""", 
"7. 业务json":"""{
    a: 1,
    b: {
        c: 3,
    }
}
""", 
"8. 包裹的json字符串":"""
```json
{
    a: 1,
    b: {
        c: 3,
    }
}
```
""", "9. 多个代码块":"""
```python
a = 3
```
```json
{
    a: 1,
    b: {
        c: 3,
    }
}
```
"""

}

for case_name, case_content in list(test_cases.items())[-3:]:
    print(f"测试用例: {case_name}")
    print(f"输入: 【{case_content}】")
    print(f"输出: 【{extract_code_content(case_content, 'json')}】")
    print("="*50)

测试用例: 7. 业务json
输入: 【{
    a: 1,
    b: {
        c: 3,
    }
}
】
输出: 【{
    a: 1,
    b: {
        c: 3,
    }
}】
测试用例: 8. 包裹的json字符串
输入: 【
```json
{
    a: 1,
    b: {
        c: 3,
    }
}
```
】
输出: 【{
    a: 1,
    b: {
        c: 3,
    }
}】
测试用例: 9. 多个代码块
输入: 【
```python
a = 3
```
```json
{
    a: 1,
    b: {
        c: 3,
    }
}
```
】
输出: 【{
    a: 1,
    b: {
        c: 3,
    }
}】


In [ ]:
# | export
import os
import asyncio
import time
from openai import AsyncOpenAI, APIError, AsyncAzureOpenAI, OpenAI, AzureOpenAI
from typing import Optional, Tuple
import os


def get_env_bool(env_var, default=False):
    env_val = os.getenv(env_var)
    if env_val is None:
        return default
    true_values = ("true", "1", "yes", "on", "y", "t")
    return env_val.strip().lower() in true_values

class Endpoint:
    def __init__(
        self,
        base_url: Optional[str] = None,
        ip: Optional[str] = None,
        port: Optional[int] = None,
        api_key: Optional[str] = None,
        model_name_or_path: Optional[str] = None,
        use_azure: Optional[bool] = None, # https://github.com/openai/openai-python
        api_version: Optional[str] = None,
    ):
        self.model_name_or_path = model_name_or_path or os.getenv("OPENAI_MODEL") or "gemini-2.5-flash"

        # Handle IPv6 addresses by wrapping them in brackets for URL compatibility
        if ip and ":" in ip:
            ip = f"[{ip}]"
        if ip and port:
            self.base_url = f"http://{ip}:{port}/v1"
        else:
            self.base_url = base_url or os.getenv("OPENAI_BASE_URL") or os.getenv("AZURE_OPENAI_ENDPOINT")
        api_key = api_key or os.getenv("OPENAI_API_KEY") or os.getenv("AZURE_OPENAI_API_KEY") or "not-needed"
        api_version = api_version or os.getenv("OPENAI_API_VERSION") or os.getenv("AZURE_OPENAI_API_VERSION") or "2024-03-01-preview"
        use_azure = use_azure or get_env_bool("OPENAI_USE_AZURE")
        if use_azure:
            self.async_client = AsyncAzureOpenAI(
                azure_endpoint=self.base_url, api_key=api_key, api_version=api_version # type: ignore
            )
            self.client = AzureOpenAI(
                azure_endpoint=self.base_url, api_key=api_key, api_version=api_version # type: ignore
            )
        else:
            self.async_client = AsyncOpenAI(base_url=self.base_url, api_key=api_key)
            self.client = OpenAI(base_url=self.base_url, api_key=api_key)

    def __repr__(self) -> str:
        return (f"Endpoint(base_url={self.base_url!r}, "
                f"model_name_or_path={self.model_name_or_path!r}, "
                f"use_azure={self.async_client.__class__.__name__.startswith('AsyncAzureOpenAI')})")

    async def chat_completions_create(self,  **kwargs):
        return await self.async_client.chat.completions.create(
            model=self.model_name_or_path,
            **kwargs
        )
    def chat_completions_create_sync(self,  **kwargs):
        return self.client.chat.completions.create(
            model=self.model_name_or_path,
            **kwargs
        )


In [45]:
from dotenv import load_dotenv

load_dotenv()
# gets the API Key from environment variable AZURE_OPENAI_API_KEY
endpoint = Endpoint()


response = await endpoint.chat_completions_create(
            messages=[
        {
            "role": "user",
            "content": "How do I output all files in a directory using Python?",
        },
    ],
        )
print(response.to_json())


{
  "id": "0217618192318445b3327adca1f53a304275db160b315eb146521",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "To output all files (including or excluding subdirectories) in a directory, you can use Python’s built-in modules like `os` (traditional) or `pathlib` (modern, object-oriented). Here are several approaches:\n\n\n### Method 1: List files in the **current directory (non-recursive)**  \nUse `os.listdir()` to get all entries, then filter for files using `os.path.isfile()`.  \n\n\n#### Using `os` module:  \n```python\nimport os\n\n# Target directory (e.g., current directory: \".\", or a specific path like \"C:/my_folder\")\ndirectory = \".\"  \n\nfor entry in os.listdir(directory):\n    full_path = os.path.join(directory, entry)\n    if os.path.isfile(full_path):\n        print(full_path)  # Print full file path\n        # print(entry)   # Uncomment to print only the filename\n```  \n\n\n#### U

In [46]:
#| export
def flatten_dict(d: dict, level: int, parent_key: str = '', sep: str = '.') -> dict:
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict) and v and level > 0:
            items.extend(flatten_dict(v, level - 1, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [47]:
# 测试 flatten_dict 函数
# 定义一个嵌套字典用于测试
test_dict = {
    'a': 1,
    'b': {
        'c': 2,
        'd': {
            'e': 3
        }
    }
}

# 测试不同的 level 值
print("level=0:", flatten_dict(test_dict, level=0))
print("level=1:", flatten_dict(test_dict, level=1))
print("level=2:", flatten_dict(test_dict, level=2))

level=0: {'a': 1, 'b': {'c': 2, 'd': {'e': 3}}}
level=1: {'a': 1, 'b.c': 2, 'b.d': {'e': 3}}
level=2: {'a': 1, 'b.c': 2, 'b.d.e': 3}
